# SEIRHVD Data Fit

This Jupyter notebooks implements the SEIRHVD model for fitting the actual Chilean data in order to make projections in VMI and bed usage, amount of infected and deaths. This model uses the SEIRHVD 4.1 version. A visual representation of the model is available at: [SEIRHVD Miró Model](https://miro.com/app/board/o9J_ktzN4xA=/)

This implements a single instance to facilitate understanding the model prior to run multiple data fittings at a time.



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
from datetime import datetime
from numpy import linalg as LA
import multiprocessing
from joblib import Parallel, delayed
from datetime import timedelta

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIRHVD/')
sys.path.insert(1, '../src/utils/')
sys.path.insert(1, 'src/SEIRHVD/')
sys.path.insert(1, 'src/utils/')

from class_SEIRHUVD6 import SEIRHVD 
from Quarantine import Quarantine
from Quarantine import SeroPrevDynamics
from importdata import ImportData
import Events

import json
from pandas.core.base import DataError
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import matplotlib

from datetime import date

import copy

Linux


## Time Variables
Declare Initial date for simulating and data fitting

In [68]:
# First simulation Initial date
initdate = datetime(2021,3,15)
# Date for change in SeroPrevalence
SPchange_date = datetime(2020,7,25)

# Current date
currentdate = datetime.now()
currentday = (currentdate - initdate).days

fiestaspatrias = datetime(2020,9,18)
fiestaspatrias_day = (fiestaspatrias-initdate).days
plebiscito = datetime(2020,10,25)
plebiscito_day = (plebiscito-initdate).days

navidad = datetime(2020,12,25)
navidad_day = (navidad-initdate).days

newyear = datetime(2020,12,31)
newyear_day = (newyear-initdate).days

today = datetime.today()
today_day = (today-initdate).days

## Import Data
To import data we use an ImportData object which is initialized with the region's cut and the initial date. state region is represented by cut = 14.

In [69]:
def dataretriever(user=None,password=None):
    # Esta funcion devolvera una funcion de request        
    def request(endpoint):
        if user:
            aux = requests.get('https://api.cv19gm.org/'+endpoint, auth=HTTPBasicAuth(user, password))
            if aux.status_code == 401:
                raise Exception('Wrong Credentials')
            else:
                print('Logged in successfully')
        else:
            aux = requests.get('http://192.168.2.223:5006/'+endpoint)
        return aux
    return request
def timeJStoPy(t):
    return datetime.strptime(t[:10],'%Y-%m-%d')
def timetxttoDatetime(t):
    return datetime.strptime(t,'%Y-%m-%d')

In [70]:
print('Importing data from all regions')
#tstate = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16']
tstate = '13'
# Import Data
state = ImportData(tstate=tstate,initdate = initdate)
state.importdata()

state.importDailyInfectedNacional()
state.name = 'Chile'

Importing data from all regions
Importing General Data
Importing Population
Importing Active infected
Importing Accumulated Infected
Importing Daily Infected
Importing ICU Beds Data from Minciencia
Importing Deaths by DEIS
Importing Active Infected by Minciencia
Done
Importing Daily Infected


### Datos de Hospitalizados a fitear

In [71]:
capacidadUCIcovid = 2600
state.Hr = 1000
state.Hr_tot = [10000,10000]
state.Vr = state.UCI_use_covid
state.Vr_tot = capacidadUCIcovid - np.array(state.UCI_use_noncovid)#[4000]*len(state.UCI_use_covid)#state.UCI_capacity
state.UCI_non_covid = state.UCI_use_noncovid
state.sochimi_tr = state.UCI_tr#[-2:]
state.sochimi_dates = state.UCI_dates

## Infectados Diarios y Acumulados

In [72]:
endpoint ='https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/TotalesPorRegion.csv'
producto3 = pd.read_csv(endpoint)
I_ac_confirmado = producto3[(producto3['Region']=='Metropolitana')&(producto3['Categoria']=='Casos acumulados')]
I_antigeno_ac = producto3[(producto3['Region']=='Metropolitana')&(producto3['Categoria']=='Casos confirmados por antigeno')]
I_probables_ac = producto3[(producto3['Region']=='Metropolitana')&(producto3['Categoria']=='Casos probables acumulados')]

In [73]:
confirmado_nac = producto3[(producto3['Region']=='Total')&(producto3['Categoria']=='Casos acumulados')].iloc[:,2:].sum()
antigeno_nac = producto3[(producto3['Region']=='Total')&(producto3['Categoria']=='Casos confirmados por antigeno')].iloc[:,2:].sum()
probables_nac = producto3[(producto3['Region']=='Total')&(producto3['Categoria']=='Casos probables acumulados')].iloc[:,2:].sum()

I_ac_nac = confirmado_nac+antigeno_nac+probables_nac
I_act_nac_roll = I_ac_nac.diff(periods=11).rolling(7).mean().dropna()


In [74]:
I_act_nac_roll.plot(label='confirmados+antigenos+probables')
confirmado_nac.diff(periods=11).rolling(7).mean().dropna().plot(label='confirmados')
plt.title('Activos')
plt.legend(loc=0)

In [75]:
R = producto3[(producto3['Region']=='Metropolitana')&(producto3['Categoria']=='Casos confirmados recuperados')].iloc[0][3:]
R.index = pd.to_datetime(R.index, format='%Y-%m-%d')
R = R.loc[R.index>=initdate]

In [76]:
initday = (initdate-pd.to_datetime(I_ac_confirmado.iloc[0].index[2], format='%Y-%m-%d')).days
#I_ac = I_ac_confirmado.iloc[0][initday:] + I_antigeno_ac.iloc[0][initday:] + I_probables_ac.iloc[0][initday:]
I_ac = I_ac_confirmado.iloc[0][3:] + I_antigeno_ac.iloc[0][3:] + I_probables_ac.iloc[0][3:]
I_ac.index = pd.to_datetime(I_ac.index, format='%Y-%m-%d')

I_d = I_ac.diff(periods=1)
I_act = I_ac.diff(periods=11)
I_d_roll = I_d.rolling(7).mean()
I_act_roll = I_act.rolling(7).mean()
I_d = I_d.loc[I_d.index>=initdate]
I_act = I_act.loc[I_act.index>=initdate]
I_ac = I_ac.loc[I_ac.index>=initdate]


I_d_roll = I_d_roll.loc[I_d_roll.index>=initdate]
I_act_roll = I_act_roll.loc[I_act_roll.index>=initdate]

state.I_d_r = I_d_roll
state.I_d_r_dates = I_d_roll.index
state.I_d_r_tr = [(I_d_roll.index[i]-initdate).days for i in range(len(I_d_roll))]

state.Ir = I_act_roll
state.Ir_dates = I_act_roll.index
state.tr =  [(I_act_roll.index[i]-initdate).days for i in range(len(I_act_roll))]

state.I_ac_r = I_ac
state.I_ac_r_dates = I_ac.index
state.I_ac_r_tr = [(I_ac.index[i]-initdate).days for i in range(len(I_ac))]



In [77]:
state.R = R

# Importante:
valores al 26 de Abril
Casos activos calculados con probables y antigenos: 38657
Casos activos confirmados publicados: 15810
Casos activos calculado solo con los casos acumulados: 28170

Activos_confirmados = producto3[(producto3['Region']=='Metropolitana')&(producto3['Categoria']=='Casos activos confirmados')]
Activos_desdeacumulados = producto3[(producto3['Region']=='Metropolitana')&(producto3['Categoria']=='Casos acumulados')].iloc[0][initday:].diff(periods=11)

## Subreport

In [78]:
Ias_det = 0.3#0.1
Imi_det = 1#0.4
Ise_det = 1
Icr_det = 1

# Parameters

In [79]:
# Total simulation time
tsim = 1000

beta = 1
mu = 0.375#5 #list(np.arange(0.2,0.4,0.01))#0.4
k_I = 0
k_R = 0#10
SeroPrevFactor = 1
expinfection=0 # Proportion in which the exposed infect - 0: nothing, 1: equally as Infected

## Quarantines

In [155]:
mob_dates = [initdate,datetime(2021,4,2),initdate+timedelta(days=tsim)]#,datetime(2021,5,3)]
mob_days = []
for i in range(len(mob_dates)-1):
    d1 = (mob_dates[i]-initdate).days
    d2 = (mob_dates[i+1]-initdate).days
    mob_days.append([d1,d2])

mobility = [0.127,0.11]

escenarios = [100]#,105,110,115,120]
alpha_var = escenarios
alpha_new = [Events.Events(values=mobility +[i*mobility[-1]/100],days=mob_days) for i in escenarios]


In [156]:
alpha_new

[<function Events.functionAddition.<locals>.f(t)>]

In [157]:
mob_days

[[0, 18], [18, 1000]]

# Plot Movilidad

In [159]:
if True:
    datelinewidth = 1
    t = np.array(np.arange(0,240,0.1))
    alphaplot = [[j(i) for i in t] for j in alpha_new]
    for f in alphaplot:
        plt.plot(t,f)
    #for f in alphaplot_UK:
        #plt.plot(t,f,linestyle='dashed') 
    plt.axvline(x = navidad_day, linestyle = 'dotted',color = 'red',label='Navidad',linewidth=datelinewidth)
    plt.axvline(x = newyear_day, linestyle = 'dotted',color = 'blue',label='Año Nuevo',linewidth=datelinewidth)
    #plt.axvline(x = mitadvacaciones_day, linestyle = 'dotted',color = 'green',label='15 Febrero',linewidth=datelinewidth)

    plt.tick_params(labelsize=18)
    plt.legend(loc=0)
    plt.title('Mobility-beta',fontsize=20)
    plt.show()


IndexError: list index out of range

## SeroPrevalence Dynamics
The susceptibles increase through a step function which last for the specified amount of days. Throughout these days it adds a "dailyincrease" amount of persons each day.

The total increase of people is increasedays*dailyincrease

In [135]:
def chisum(a,b,c):
    def aux(t):
        return a(t)+b(t)+c(t)
    return aux

In [136]:
def aux(t):
    return 0
chi = aux

In [137]:
chi = []

In [138]:
increasedays = 10
renewalFactor = -0.1
dailyincrease = state.population*SeroPrevFactor*renewalFactor/increasedays
t_sp_temp = (datetime(2021,4,2)-initdate).days
chi0 = SeroPrevDynamics(t_sp_temp,t_sp_temp+increasedays*0.8,t_sp_temp+increasedays,dailyincrease,form='line')

In [139]:
increasedays = 9
renewalFactor = 0
dailyincrease = state.population*SeroPrevFactor*renewalFactor/increasedays
t_sp_temp = (datetime(2021,4,11)-initdate).days
chi1 = SeroPrevDynamics(t_sp_temp,t_sp_temp+increasedays*0.8,t_sp_temp+increasedays,dailyincrease,form='line')

In [140]:
increasedays = 10
renewalFactor = 0
dailyincrease = state.population*SeroPrevFactor*renewalFactor/increasedays
t_sp_temp = (datetime(2021,5,3)-initdate).days
chi2 = SeroPrevDynamics(t_sp_temp,t_sp_temp+increasedays*0.8,t_sp_temp+increasedays,dailyincrease,form='line')

In [141]:
chi = chisum(chi0,chi1,chi2)

In [142]:
chiplot = [chi(t) for t in np.arange(0,tsim,0.1)]

In [143]:
#plt.plot(chiplot)

#### Plot Susceptible Dynamics

# State transition parameters:

#state parameters: 
pE_Ias = 0.43 # Transition from exposed to Asymptomatic Infected
tE_Ias = 3.0
pE_Imi = 0.53 # Transition from exposed to  Mild Infected
tE_Imi = 3.0
pE_Icr = 0.0065 # Transition from exposed to  Critical Infected
tE_Icr = 3.0
pE_Ise = 0.0335# Transition from exposed to  Serious Infected
tE_Ise = 3.0
pIas_R = 1.0   # Transition from Asymptomatic Infected to Recovered
tIas_R = 5.0
pImi_R = 1.0  # Transition from Mild Infected to Recovered
tImi_R = 5.0
pIse_Hse = 1 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)
tIse_Hse = 5.0
pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)
tIse_D = 3.0
pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)
tIcr_V = 3.0
pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)
tIcr_D = 3.0
pHse_R = 0.97 # Transition from Serious Hospitalized to Recovered
tHse_R = 3.5
pHse_V = 0.03# Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)
tHse_V = 4.0
pHse_D = 0.85# Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)
tHse_D = 4.0
pV_Hout = 0.45  # Transition from Ventilators to Hospital Recovery (Hout)
tV_Hout = 16.0
pV_D = 0.55 # Transition from Ventilators to Death
tV_D = 12.0
pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered
tHout_R = 10.0

In [144]:
#state parameters: 
pE_Ias = 0.30 # Transition from exposed to Asymptomatic Infected
tE_Ias = 3.0
pE_Imi = 0.55 # Transition from exposed to  Mild Infected
tE_Imi = 3.0
pE_Icr = 0.04 # Transition from exposed to  Critical Infected
tE_Icr = 3.0
pE_Ise = 0.11 # Transition from exposed to  Serious Infected
tE_Ise = 3.0
pIas_R = 1.0  # Transition from Asymptomatic Infected to Recovered
tIas_R = 12.0
pImi_R = 1.0  # Transition from Mild Infected to Recovered
tImi_R = 12.0
pIse_Hse = 1 # Transition from Serious Infected to Serious Hospitalized (When Hospital capacity is not saturated)
tIse_Hse = 11.0
pIse_D = 1.0  # Transition from Serious Infected to Death (When Hospital capacity is saturated)
tIse_D = 11.0
pIcr_V = 1.0  # Transition from Critical Infected to Ventilator (When Ventilators capacity is not saturated)
tIcr_V = 7.0
pIcr_D = 1.0  # Transition from Serious Infected to Death (When Ventilators capacity is saturated)
tIcr_D = 7.0
pHse_R = 0.97 # Transition from Serious Hospitalized to Recovered
tHse_R = 3.5
pHse_V = 0.03# Transition from Serious Hospitalized to Ventilators (When Ventilators capacity is not saturated)
tHse_V = 4.0
pHse_D = 0.85# Transition from Serious Hospitalized to Death (When Ventilators capacity is saturated)
tHse_D = 4.0
pV_Hout = 0.7  # Transition from Ventilators to Hospital Recovery (Hout)
tV_Hout = 11.0
pV_D = 0.3 # Transition from Ventilators to Death
tV_D = 11.0
pHout_R = 1.0 # Transition from Hospital Recovery (Hout) to Recovered
tHout_R = 10.0

### Tiempo medio Infeccioso

In [145]:
pE_Ias+pE_Imi+pE_Ise+pE_Icr

1.0

In [146]:
pE_Ias*tIas_R + pE_Imi*tImi_R  + pE_Ise*tIse_Hse + pE_Icr*tIcr_V

11.69

# Simulation

## Initial SeroPrev period
Initialize Simulation Object, set params and run the simulation. 

In [147]:
simulation = [SEIRHVD(tsim,beta,mu,alpha_new[i],k_I=k_I,k_R = k_R, chi = chi,                      
                      SeroPrevFactor=SeroPrevFactor,expinfection=expinfection,initdate = initdate,
                      Imi_det = Imi_det,Ias_det = Ias_det,RealIC=state) for i in range(len(alpha_new))]

InitialCondition Object Data


Modify simulation state parameters

In [148]:
for i in range(len(simulation)):
    simulation[i].pE_Ias=pE_Ias
    simulation[i].tE_Ias=tE_Ias
    simulation[i].pE_Imi=pE_Imi
    simulation[i].tE_Imi=tE_Imi
    simulation[i].pE_Icr=pE_Icr
    simulation[i].tE_Icr=tE_Icr
    simulation[i].pE_Ise=pE_Ise
    simulation[i].tE_Ise=tE_Ise
    simulation[i].pIas_R=pIas_R
    simulation[i].tIas_R =tIas_R
    simulation[i].pImi_R=pImi_R
    simulation[i].tImi_R =tImi_R
    simulation[i].pIse_Hse=pIse_Hse
    simulation[i].tIse_Hse=tIse_Hse
    simulation[i].pIse_D=pIse_D
    simulation[i].tIse_D=tIse_D
    simulation[i].pIcr_V=pIcr_V
    simulation[i].tIcr_V=tIcr_V
    simulation[i].pIcr_D=pIcr_D
    simulation[i].tIcr_D=tIcr_D
    simulation[i].pHse_R=pHse_R
    simulation[i].tHse_R =tHse_R
    simulation[i].pHse_V=pHse_V
    simulation[i].tHse_V=tHse_V
    simulation[i].pHse_D=pHse_D
    simulation[i].tHse_D=tHse_D
    simulation[i].pV_Hout=pV_Hout
    simulation[i].tV_Hout =tV_Hout
    simulation[i].pV_D=pV_D
    simulation[i].tV_D =tV_D
    simulation[i].pHout_R=pHout_R
    simulation[i].tHout_R=tHout_R
    simulation[i].setnewparams()

Compartimental model State parameters changed


### Setting new params

## Run simulation

* **integr_sci**: Fast solver, sometimes stiffness beat it  
* **integr:** Slow solver, more robust with stiffness  

In [149]:
def simulate(simulation,i,tsim):
    simulation[i].integr_sci(0,tsim,0.1)
    return simulation[i]

simulation[0].integr_sci(0,1000,0.1)

In [150]:
num_cores = multiprocessing.cpu_count()
sims = Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(simulation,i,tsim) for i in range(len(simulation)))
simulation = sims
print('ready')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.1s


IndexError: list index out of range

# Analysis

In [ ]:
saveplot = False
if saveplot:
    %matplotlib inline
#%matplotlib inline
%matplotlib tk
# Big Screen
plt.rcParams["figure.figsize"] = 38.40, 20.56 #19.20,10.80#8,6
# Small Screen
#plt.rcParams["figure.figsize"] = 19.2, 10.80 #19.20,10.80#8,6
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "sans-serif",
    "font.sans-serif":["Arial"]})


# Plot

In [113]:
# Days to plot
days = 500
t_end = 50

enddate = initdate + timedelta(days=365)
enddate = datetime(2021,6,15)


dates = [[initdate+timedelta(days=int(i)) for i in simulation[j].t] for j in range(len(simulation))]
SeroPrevalence = [simulation[i].I_ac[-1]/simulation[i].population for i in range(len(simulation))]
SeroPrevalenceDet = [simulation[i].I_ac_det[-1]/simulation[i].population for i in range(len(simulation))]

In [103]:
simulation[0].dates[-1]

datetime.datetime(2023, 12, 9, 0, 0)

### Pantalla

In [104]:
pantalla = '1080'
if pantalla == '4k':
    fontsize_title = 22
    fontsize_suptitle = 25
    fontsize_label = 18
    fontsize_legend = 14

#1080
elif pantalla == '1080':
    fontsize_title = 15
    fontsize_suptitle = 16
    fontsize_label = 10
    fontsize_legend = 12

pĺotlinewidth = 2
datelinewidth = 1.5

# Grid Plot

In [105]:
renewalFactor = [0]
sim_InfDiario = False
sim_InfEPI = True

colors = plt.cm.rainbow_r(np.linspace(0,1,len(simulation)))

fig, axs = plt.subplots(2, 2)


for i in range(len(simulation)):
    if type(mu) == list:
        axs[0,0].plot(dates[i],simulation[i].I_det,linewidth=pĺotlinewidth,label='mu: '+str(mu[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
    if type(alpha_new) == list:
        axs[0,0].plot(dates[i],simulation[i].I_det,linewidth=pĺotlinewidth,label='Movilidad: '+str(escenarios[i])+'%',color=colors[i])
axs[0,0].set_title('Infectados Activos Detectados',fontsize=fontsize_title)

axs[0,0].scatter(state.Ir_dates,state.Ir,color='grey',label='DP3 med_mov 7',zorder=10)


# Accumulated deaths
for i in range(len(simulation)):
    if type(mu) == list:
        axs[0,1].plot(dates[i],simulation[i].B,label='mu: '+str(mu[i]),color=colors[i],linewidth=pĺotlinewidth)
    if type(alpha_new) == list:
        axs[0,1].plot(dates[i],simulation[i].B,label='Movilidad: '+str(escenarios[i])+'%',color=colors[i],linewidth=pĺotlinewidth)
    
axs[0,1].set_title('Fallecidos Acumulados (PCR+)',fontsize=fontsize_title)

axs[0,1].scatter(state.Br_dates,state.Br,label='DP50',color='grey',zorder=10)


for i in range(len(simulation)):
    if type(mu) == list:
        axs[1,0].plot(dates[i],simulation[i].I_d_det,label='mu: '+str(mu[i]),color=colors[i],linewidth=pĺotlinewidth)
    if type(alpha_new) == list:
        axs[1,0].plot(dates[i],simulation[i].I_d_det,label='Movilidad: '+str(escenarios[i])+'%',color=colors[i],linewidth=pĺotlinewidth)
axs[1,0].set_title('Nuevos Infectados Diarios Detectados',fontsize=fontsize_title)


if False:#sim_InfEPI:
    axs[1,0].scatter(state.I_d_r_dates,state.I_d_r,label='Informe EPI',color='black',zorder=5)
if sim_InfDiario:
    axs[1,0].scatter(I_week_ra_dates,I_week_ra,label='Informe diario (RA5D)',color='red',zorder=5)  
    axs[1,0].plot(dates_I_d,I_d,label='Informe diario',color='blue',zorder=10,linestyle='dashed')

#axs[1,0].scatter(state.I_d_r_dates,state.I_d_r,label='DP5',color='black',zorder=5)
axs[1,0].scatter(state.I_d_r_dates,I_d_roll,label='DP3 med_mov 7',color='grey',zorder=5)
    

axs[1,0].legend(loc='upper left')

axs[1,1].plot(state.UCI_dates,capacidadUCIcovid-np.array(state.UCI_use_noncovid),label='Limite uso COVID',color='black',zorder=10)
for i in range(len(simulation)):
    if type(mu) == list:
        axs[1,1].plot(dates[i],simulation[i].V,label='mu: '+str(mu[i]),color=colors[i],linewidth=pĺotlinewidth)
    #axs[1,1].plot(dates[i],simulation[i].V_need,linestyle='dashed',color=colors[i])
    if type(alpha_new) == list:
        axs[1,1].plot(dates[i],simulation[i].V,label='Mov: '+str(escenarios[i])+'%',color=colors[i],linewidth=pĺotlinewidth)
axs[1,1].set_title('Uso UCI Covid',fontsize=fontsize_title)

axs[1,1].scatter(state.UCI_dates,state.UCI_use_covid,label='DP58',color='grey',zorder=10)
axs[1,1].plot(simulation[0].dates,simulation[0].V_cap, linestyle = 'dashed', color = 'black')


# Ephemerides
for i in mob_dates:
    axs[0,0].axvline(x = i, linestyle = 'dashed',linewidth=datelinewidth)
    axs[0,1].axvline(x = i, linestyle = 'dashed',linewidth=datelinewidth)
    axs[1,0].axvline(x = i, linestyle = 'dashed',linewidth=datelinewidth)
    axs[1,1].axvline(x = i, linestyle = 'dashed',linewidth=datelinewidth)
 
axs[0,0].axvline(x = datetime(2021,4,11), linestyle = 'dashed',linewidth=datelinewidth,color='red')
axs[0,0].axvline(x = datetime(2021,5,3), linestyle = 'dashed',linewidth=datelinewidth,color='grey')
axs[0,1].axvline(x = datetime(2021,4,11), linestyle = 'dashed',linewidth=datelinewidth,color='red')
axs[0,1].axvline(x = datetime(2021,5,3), linestyle = 'dashed',linewidth=datelinewidth,color='grey')
axs[1,0].axvline(x = datetime(2021,4,11), linestyle = 'dashed',linewidth=datelinewidth,color='red')
axs[1,0].axvline(x = datetime(2021,5,3), linestyle = 'dashed',linewidth=datelinewidth,color='grey')
axs[1,1].axvline(x = datetime(2021,4,11), linestyle = 'dashed',linewidth=datelinewidth,color='red')
axs[1,1].axvline(x = datetime(2021,5,3), linestyle = 'dashed',linewidth=datelinewidth,color='grey')

# Format and shit

# Grid
axs[0,0].grid(color='grey', linestyle='dotted', linewidth=0.5)
axs[0,1].grid(color='grey', linestyle='dotted', linewidth=0.5)
axs[1,0].grid(color='grey', linestyle='dotted', linewidth=0.5)
axs[1,1].grid(color='grey', linestyle='dotted', linewidth=0.5)

# x axis: Date range
axs[0,0].set_xlim(initdate,enddate)
axs[0,1].set_xlim(initdate,enddate)
axs[1,0].set_xlim(initdate,enddate)
axs[1,1].set_xlim(initdate,enddate)

# x axis: Date interval
interval = 30
#axs[0,0].xaxis.set_major_locator(mdates.DayLocator(interval=interval))
#axs[0,1].xaxis.set_major_locator(mdates.DayLocator(interval=interval))
#axs[1,0].xaxis.set_major_locator(mdates.DayLocator(interval=interval))
#axs[1,1].xaxis.set_major_locator(mdates.DayLocator(interval=interval))

# x axis: Date format
axs[0,0].get_xaxis().set_major_formatter(mdates.DateFormatter('%d-%m'))
axs[0,1].get_xaxis().set_major_formatter(mdates.DateFormatter('%d-%m'))
axs[1,0].get_xaxis().set_major_formatter(mdates.DateFormatter('%d-%m'))
axs[1,1].get_xaxis().set_major_formatter(mdates.DateFormatter('%d-%m'))

# Y axis: Format
axs[0,0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
axs[0,1].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
axs[1,0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
axs[1,1].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

# Y axis: Range
#axs[0,0].set_ylim(0,5e4)
axs[0,1].set_ylim(0,50000)
#axs[1,0].set_ylim(0,10000)
#axs[1,1].set_ylim(0,2.5e3)
#axs[0,0].set_yscale('log')

# Font-size
axs[0,0].tick_params(labelsize=fontsize_label)
axs[0,1].tick_params(labelsize=fontsize_label)
axs[1,0].tick_params(labelsize=fontsize_label)
axs[1,1].tick_params(labelsize=fontsize_label)

# Legend
axs[0,0].legend(loc='lower right',fontsize=fontsize_legend)
axs[0,1].legend(loc='lower right',fontsize=fontsize_legend)
axs[1,0].legend(loc='lower right',fontsize=fontsize_legend)
axs[1,1].legend(loc='lower right',fontsize=fontsize_legend)
#axs[1,1].legend(bbox_to_anchor=(0.33, 0.52),fontsize=fontsize_legend)


fig.suptitle(title,fontsize=fontsize_suptitle)
fig.show()

In [106]:
renewalFactor = [0]
sim_InfDiario = False
sim_InfEPI = True

colors = plt.cm.rainbow_r(np.linspace(0,1,len(simulation)))

fig, axs = plt.subplots(2, 2)


for i in range(len(simulation)):
    if type(mu) == list:
        axs[0,0].plot(dates[i],simulation[i].I_det,linewidth=pĺotlinewidth,label='mu: '+str(mu[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
    if type(alpha_new) == list:
        axs[0,0].plot(dates[i],simulation[i].I_det,linewidth=pĺotlinewidth,label='Movilidad: '+str(escenarios[i])+'%',color=colors[i])
axs[0,0].set_title('Infectados Activos Detectados',fontsize=fontsize_title)

axs[0,0].scatter(state.Ir_dates,state.Ir,color='grey',label='DP3 med_mov 7',zorder=10)


# Accumulated deaths
for i in range(len(simulation)):
    if type(mu) == list:
        axs[0,1].plot(dates[i],simulation[i].B,label='mu: '+str(mu[i]),color=colors[i],linewidth=pĺotlinewidth)
    if type(alpha_new) == list:
        axs[0,1].plot(dates[i],simulation[i].B,label='Movilidad: '+str(escenarios[i])+'%',color=colors[i],linewidth=pĺotlinewidth)
    
axs[0,1].set_title('Fallecidos Acumulados (PCR+)',fontsize=fontsize_title)

axs[0,1].scatter(state.Br_dates,state.Br,label='DP50',color='grey',zorder=10)


for i in range(len(simulation)):
    if type(mu) == list:
        axs[1,0].plot(dates[i],simulation[i].I_d_det,label='mu: '+str(mu[i]),color=colors[i],linewidth=pĺotlinewidth)
    if type(alpha_new) == list:
        axs[1,0].plot(dates[i],simulation[i].I_d_det,label='Movilidad: '+str(escenarios[i])+'%',color=colors[i],linewidth=pĺotlinewidth)
axs[1,0].set_title('Nuevos Infectados Diarios Detectados',fontsize=fontsize_title)


if False:#sim_InfEPI:
    axs[1,0].scatter(state.I_d_r_dates,state.I_d_r,label='Informe EPI',color='black',zorder=5)
if sim_InfDiario:
    axs[1,0].scatter(I_week_ra_dates,I_week_ra,label='Informe diario (RA5D)',color='red',zorder=5)  
    axs[1,0].plot(dates_I_d,I_d,label='Informe diario',color='blue',zorder=10,linestyle='dashed')

#axs[1,0].scatter(state.I_d_r_dates,state.I_d_r,label='DP5',color='black',zorder=5)
axs[1,0].scatter(state.I_d_r_dates,I_d_roll,label='DP3 med_mov 7',color='grey',zorder=5)
    

axs[1,0].legend(loc='upper left')

axs[1,1].plot(state.UCI_dates,capacidadUCIcovid-np.array(state.UCI_use_noncovid),label='Limite uso COVID',color='black',zorder=10)
for i in range(len(simulation)):
    if type(mu) == list:
        axs[1,1].plot(dates[i],simulation[i].V,label='mu: '+str(mu[i]),color=colors[i],linewidth=pĺotlinewidth)
    #axs[1,1].plot(dates[i],simulation[i].V_need,linestyle='dashed',color=colors[i])
    if type(alpha_new) == list:
        axs[1,1].plot(dates[i],simulation[i].V,label='Mov: '+str(escenarios[i])+'%',color=colors[i],linewidth=pĺotlinewidth)
axs[1,1].set_title('Uso UCI Covid',fontsize=fontsize_title)

axs[1,1].scatter(state.UCI_dates,state.UCI_use_covid,label='DP58',color='grey',zorder=10)
axs[1,1].plot(simulation[0].dates,simulation[0].V_cap, linestyle = 'dashed', color = 'black')


# Ephemerides
for i in mob_dates:
    axs[0,0].axvline(x = i, linestyle = 'dashed',linewidth=datelinewidth)
    axs[0,1].axvline(x = i, linestyle = 'dashed',linewidth=datelinewidth)
    axs[1,0].axvline(x = i, linestyle = 'dashed',linewidth=datelinewidth)
    axs[1,1].axvline(x = i, linestyle = 'dashed',linewidth=datelinewidth)
 
axs[0,0].axvline(x = datetime(2021,4,11), linestyle = 'dashed',linewidth=datelinewidth,color='red')
axs[0,0].axvline(x = datetime(2021,5,3), linestyle = 'dashed',linewidth=datelinewidth,color='grey')
axs[0,1].axvline(x = datetime(2021,4,11), linestyle = 'dashed',linewidth=datelinewidth,color='red')
axs[0,1].axvline(x = datetime(2021,5,3), linestyle = 'dashed',linewidth=datelinewidth,color='grey')
axs[1,0].axvline(x = datetime(2021,4,11), linestyle = 'dashed',linewidth=datelinewidth,color='red')
axs[1,0].axvline(x = datetime(2021,5,3), linestyle = 'dashed',linewidth=datelinewidth,color='grey')
axs[1,1].axvline(x = datetime(2021,4,11), linestyle = 'dashed',linewidth=datelinewidth,color='red')
axs[1,1].axvline(x = datetime(2021,5,3), linestyle = 'dashed',linewidth=datelinewidth,color='grey')

# Format and shit

# Grid
axs[0,0].grid(color='grey', linestyle='dotted', linewidth=0.5)
axs[0,1].grid(color='grey', linestyle='dotted', linewidth=0.5)
axs[1,0].grid(color='grey', linestyle='dotted', linewidth=0.5)
axs[1,1].grid(color='grey', linestyle='dotted', linewidth=0.5)

# x axis: Date range
axs[0,0].set_xlim(initdate,enddate)
axs[0,1].set_xlim(initdate,enddate)
axs[1,0].set_xlim(initdate,enddate)
axs[1,1].set_xlim(initdate,enddate)

# x axis: Date interval
interval = 30
#axs[0,0].xaxis.set_major_locator(mdates.DayLocator(interval=interval))
#axs[0,1].xaxis.set_major_locator(mdates.DayLocator(interval=interval))
#axs[1,0].xaxis.set_major_locator(mdates.DayLocator(interval=interval))
#axs[1,1].xaxis.set_major_locator(mdates.DayLocator(interval=interval))

# x axis: Date format
axs[0,0].get_xaxis().set_major_formatter(mdates.DateFormatter('%d-%m'))
axs[0,1].get_xaxis().set_major_formatter(mdates.DateFormatter('%d-%m'))
axs[1,0].get_xaxis().set_major_formatter(mdates.DateFormatter('%d-%m'))
axs[1,1].get_xaxis().set_major_formatter(mdates.DateFormatter('%d-%m'))

# Y axis: Format
axs[0,0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
axs[0,1].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
axs[1,0].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
axs[1,1].get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

# Y axis: Range
#axs[0,0].set_ylim(0,5e4)
axs[0,1].set_ylim(0,50000)
#axs[1,0].set_ylim(0,10000)
#axs[1,1].set_ylim(0,2.5e3)
#axs[0,0].set_yscale('log')

# Font-size
axs[0,0].tick_params(labelsize=fontsize_label)
axs[0,1].tick_params(labelsize=fontsize_label)
axs[1,0].tick_params(labelsize=fontsize_label)
axs[1,1].tick_params(labelsize=fontsize_label)

# Legend
axs[0,0].legend(loc='lower right',fontsize=fontsize_legend)
axs[0,1].legend(loc='lower right',fontsize=fontsize_legend)
axs[1,0].legend(loc='lower right',fontsize=fontsize_legend)
axs[1,1].legend(loc='lower right',fontsize=fontsize_legend)
#axs[1,1].legend(bbox_to_anchor=(0.33, 0.52),fontsize=fontsize_legend)


fig.suptitle(title,fontsize=fontsize_suptitle)
fig.show()

In [126]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(simulation)))

fig, axs = plt.subplots(1, 1)

axs.scatter(state.Ir_dates,state.Ir,color='grey',label='DP3 med_mov 7',zorder=10)

axs.set_ylim(0,70000)
axs.legend(loc='lower right',fontsize=fontsize_legend)
axs.set_title('Infectados Activos Detectados',fontsize=fontsize_title)



Text(0.5, 1.0, 'Infectados Activos Detectados')

In [160]:
renewalFactor = [0]
sim_InfDiario = False
sim_InfEPI = True

colors = plt.cm.rainbow_r(np.linspace(0,1,len(simulation)))

fig, axs = plt.subplots(1, 1)

axs.scatter(state.Ir_dates,state.Ir,color='grey',label='DP3 med_mov 7',zorder=10)

for i in range(len(simulation)):
    if type(mu) == list:
        axs.plot(dates[i],simulation[i].I_det,linewidth=pĺotlinewidth,label='mu: '+str(mu[i])+' | SP_R: '+str(round(100*SeroPrevalence[i],2))+'%'+' | SP_D:'+str(round(100*SeroPrevalenceDet[i],2))+'%',color=colors[i])
    if type(alpha_new) == list:
        axs.plot(dates[i],simulation[i].I_det,linewidth=pĺotlinewidth,label='Simulation',color=colors[i])
axs.set_title('Infectados Activos Detectados',fontsize=fontsize_title)


# Grid
axs.grid(color='grey', linestyle='dotted', linewidth=0.5)


# x axis: Date range
axs.set_xlim(initdate,enddate)


# x axis: Date interval
interval = 30

# x axis: Date format
axs.get_xaxis().set_major_formatter(mdates.DateFormatter('%d-%m'))

# Y axis: Format
axs.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

# Y axis: Range
axs.set_ylim(0,70000)

# Font-size
axs.tick_params(labelsize=fontsize_label)
# Legend
axs.legend(loc='lower right',fontsize=fontsize_legend)

fig.suptitle(title,fontsize=fontsize_suptitle)
fig.show()

AttributeError: 'SEIRHVD' object has no attribute 'I_det'

In [69]:
daterange = [initdate_table+timedelta(days=7*i) for i in range(int((enddate-initdate_table).days/7)+1)]

NameError: name 'initdate_table' is not defined

In [ ]:
int((enddate-initdate).days/7)

# Generación de tablas

In [70]:
initdate_table = datetime(2021,5,1)
enddate_table = datetime(2021,12,31)

init_index = np.where(np.array(dates[0]) == np.array(initdate_table))[0][0]
end_index = np.where(np.array(dates[0]) == np.array(enddate_table))[0][0] + 1 

Construir DataFrame

In [71]:
np.around(np.array([1.1,2.5,8.9])).astype(int)

array([1, 2, 9])

In [72]:
vmi_dict = [{'Uso_UCI_mov_'+str(escenarios[i]):np.around(simulation[i].V[init_index:end_index]).astype(int)} for i in range(len(escenarios))]
I_d_dict = [{'Infectados_diarios_mov_'+str(escenarios[i]):np.around(simulation[i].I_d_det[init_index:end_index]).astype(int)} for i in range(len(escenarios))]
I_act_dict = [{'Infectados_activos_mov_'+str(escenarios[i]):np.around(simulation[i].I_det[init_index:end_index]).astype(int)} for i in range(len(escenarios))]
D_acc_dict = [{'Fallecidos_acumulados_mov_'+str(escenarios[i]):np.around(simulation[i].B[init_index:end_index]).astype(int)} for i in range(len(escenarios))]

In [73]:
vmi_table = {'fechas':dates[0][init_index:end_index]}
I_d_table = {'fechas':dates[0][init_index:end_index]}
I_act_table = {'fechas':dates[0][init_index:end_index]}
D_acc_table = {'fechas':dates[0][init_index:end_index]}
for i in vmi_dict:
    vmi_table =  {**vmi_table, **i}
for i in I_d_dict:    
    I_d_table =  {**I_d_table, **i}    
for i in I_act_dict:    
    I_act_table =  {**I_act_table, **i}    
for i in D_acc_dict:
    D_acc_table =  {**D_acc_table, **i}

In [74]:
vmi_df = pd.DataFrame(vmi_table)
I_d_df = pd.DataFrame(I_d_table)
I_act_df = pd.DataFrame(I_act_table)
D_acc_df = pd.DataFrame(D_acc_table)

In [75]:
D_acc_df

,fechas,Fallecidos_acumulados_mov_100,Fallecidos_acumulados_mov_105,Fallecidos_acumulados_mov_110,Fallecidos_acumulados_mov_115,Fallecidos_acumulados_mov_120
0,2021-05-01,14567,14567,14567,14567,14567
1,2021-05-02,14620,14620,14620,14620,14620
2,2021-05-03,14672,14672,14672,14672,14672
3,2021-05-04,14724,14724,14724,14724,14724
4,2021-05-05,14775,14775,14775,14775,14775
...,...,...,...,...,...,...
240,2021-12-27,38734,74999,115955,157080,196804
241,2021-12-28,38784,75170,116251,157420,197108
242,2021-12-29,38833,75338,116542,157751,197402
243,2021-12-30,38882,75501,116827,158074,197685


In [76]:
vmi_df.to_csv('/home/samuel/Dropbox/DLab/Covid/DataProjections/Chile/UCI_may-dic_0206.csv')
I_d_df.to_csv('/home/samuel/Dropbox/DLab/Covid/DataProjections/Chile/I_diarios_may-dic_0206.csv')
I_act_df.to_csv('/home/samuel/Dropbox/DLab/Covid/DataProjections/Chile/I_activos_may-dic_0206.csv')
D_acc_df.to_csv('/home/samuel/Dropbox/DLab/Covid/DataProjections/Chile/D_accu_may-dic_0206.csv')

#### infectadios diarios

In [ ]:
i=0
plt.plot(dates[i],simulation[i].I_d)
plt.show()

In [ ]:
Id_dict = [{'Id_mov_'+str(escenarios[i]):np.around(simulation[i].I_d_det)} for i in range(len(escenarios))]

In [ ]:
Id_table = {'fechas':dates[0]}
for i in Id_dict:
    Id_table =  {**Id_table, **i} 
Id_df = pd.DataFrame(Id_table)

In [ ]:
Id_10may = Id_df[(Id_df['fechas']>=datetime(2021,5,10)) & (Id_df['fechas']<datetime(2021,5,17))]

In [ ]:
Id_10may

In [ ]:
Id_average = [np.mean(Id_10may[i]) for i in Id_10may.keys()[1:]]

In [ ]:
np.around(Id_average)

In [ ]:
Id_10may